In [56]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten,Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os 
import cv2
import matplotlib.pyplot as plt

**Load data**

In [ ]:
data_dir='/kaggle/input/intel-image-classification'

files_name =os.listdir(data_dir)
class_name =os.listdir(os.path.join(data_dir,files_name[0],files_name[0]))
class_name

**load training dataset**

In [ ]:
train_dir=os.path.join(data_dir,files_name[0],files_name[0])
data=tf.keras.utils.image_dataset_from_directory(train_dir,image_size=(100, 100),seed=42)

**show your data**

In [ ]:
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()
class_name

In [ ]:
fig, aix =plt.subplots(ncols=5,figsize=(20,20))
for ind,img in enumerate(batch[0][:5]):
    aix[ind].imshow(img.astype(int))
    aix[ind].title.set_text(batch[1][ind])

**load test and predict dataset**

In [ ]:
test_dir=os.path.join(data_dir,files_name[2],files_name[2])
test_data=tf.keras.utils.image_dataset_from_directory(test_dir,image_size=(100, 100))

**split data**

In [ ]:
train_size=int(len(data)*.8)
val_size =int(len(data)*.2)+1

In [ ]:
train_data=data.take(train_size)
val_data=data.skip(train_size).take(val_size)

**scale data**

In [ ]:
train_data =train_data.map(lambda x,y:(x/255,y) )
val_data=val_data.map(lambda x,y:(x/255,y) )

**Build a deep learing model**

In [ ]:
model=Sequential()

In [ ]:
model.add(Conv2D(100,(3,3),activation='relu',kernel_regularizer=regularizers.l2(0.01),input_shape=(100,100,3)))
model.add(MaxPooling2D())

model.add(Conv2D(80,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(50,(3,3),activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(40,(3,3),activation='relu'))
model.add(MaxPooling2D())


model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.2))

model.add(Dense(6,activation='softmax'))

**now to compile the model , using adam optimizer , & sparse categorical crossentropy loss**

In [ ]:
model.compile(optimizer ='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
hist=model.fit(train_data,epochs=20,validation_data=val_data)

In [ ]:
fig=plt.figure()
plt.plot(hist.history['loss'],label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')
fig.suptitle('loss',fontsize=20)
plt.legend()
plt.show()

In [ ]:
fig =plt.figure()
plt.plot(hist.history['accuracy'],label='accuracy')
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
fig.suptitle('accuracy',fontsize=20)
plt.legend()
plt.show()

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
from tensorflow.keras.utils import to_categorical
for batch in test_data.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    y_one_hot = to_categorical(y, num_classes=6)
    pre.update_state(y_one_hot, yhat)
    re.update_state(y_one_hot, yhat)
    acc.update_state(y_one_hot, yhat)

In [ ]:
print(f'Precision= {pre.result().numpy()}, Recall= {re.result().numpy()}, Accuracy= {acc.result().numpy()}')

In [ ]:
model.save('v.h5')